In [206]:
import pandas as pd

In [220]:
df = pd.read_csv("../data/out/combined_data.csv")
df.head(500)

,Years,Number,country
0,2004,1332,France
1,2005,1212,France
2,2006,1211,France
3,2007,1161,France
4,2008,1331,France
...,...,...,...
495,Source: ECIS - European Cancer Information System,NaN,Austria
496,From https://ecis.jrc.ec.europa.eu/,accessed on 5/9/2024,Austria
497,(C) European union,"2024""",Austria
498,2004,1482,Belgium


In [221]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1541 entries, 0 to 1540
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Years    1541 non-null   object
 1   Number   1406 non-null   object
 2   country  1541 non-null   object
dtypes: object(3)
memory usage: 36.2+ KB


In [222]:
df.describe()

,Years,Number,country
count,1541,1406,1541
unique,13,247,23
top,2008,accessed on 5/9/2024,France
freq,135,135,704


In [223]:
df.nunique()

Years       13
Number     247
country     23
dtype: int64

In [236]:
df['Number'] = pd.to_numeric(df['Number'], errors='coerce')

In [273]:
df_cleaned = df[~df['Years'].isin(["(C) European union", "From https://ecis.jrc.ec.europa.eu/", "Source: ECIS - European Cancer Information System"])]
df_cleaned.reset_index(drop=True, inplace=True)

In [274]:
df_fr_de = df_cleaned[(df_cleaned["country"] == "Germany") | (df_cleaned["country"] == "France")]
df_fr_de.head(300)

,Years,Number,country
0,2004,1332.0,France
1,2005,1212.0,France
2,2006,1211.0,France
3,2007,1161.0,France
4,2008,1331.0,France
...,...,...,...
655,2013,345.0,France
656,2008,2210.0,France
657,2009,2279.0,France
658,2010,2250.0,France


In [259]:
df_grouped_fr_de = df_fr_de.groupby(['Years', 'country'])["Number"].mean().reset_index().drop_duplicates(subset=["country", "Years"], keep='first')
df_grouped_fr_de.head(25)

,Years,country,Number
0,2004,France,598.607843
1,2004,Germany,853.285714
2,2005,France,619.352941
3,2005,Germany,886.714286
4,2006,France,625.803922
5,2006,Germany,883.142857
6,2007,France,637.886792
7,2007,Germany,923.714286
8,2008,France,746.241935
9,2008,Germany,1594.300000


In [315]:
df_other_countries = df_cleaned[(df_cleaned["country"] != "Germany") & (df_cleaned["country"] != "France")].reset_index(drop=True)
df_other_countries.head(900)

,Years,Number,country
0,2004,1332.0,Austria
1,2005,1212.0,Austria
2,2006,1211.0,Austria
3,2007,1161.0,Austria
4,2008,1331.0,Austria
...,...,...,...
535,2008,85.0,Spain
536,2009,96.0,Spain
537,2010,114.0,Spain
538,2011,117.0,Spain


In [319]:
df_grouped_other = df_other_countries.groupby(['Years', 'country'])["Number"].mean().reset_index().drop_duplicates(subset=["country", "Years"]).reset_index(drop=True)
df_grouped_other.count()

Years      180
country    180
Number     180
dtype: int64

In [320]:
#df_grouped_fr_de.loc[(df_grouped_fr_de["Years"] == 2013) & (df_grouped_fr_de["country"] == "France")] = None
df_concat = pd.concat([df_grouped_other, df_grouped_fr_de])
df_concat.head(10)

,Years,country,Number
0,2004,Austria,1332.0
1,2004,Belgium,1482.0
2,2004,Bulgaria,409.0
3,2004,Croatia,438.0
4,2004,Cyprus,28.0
5,2004,Czechia,409.0
6,2004,Denmark,409.0
7,2004,Estonia,144.0
8,2004,Iceland,53.0
9,2004,Ireland,555.0


In [323]:
df_pivot = df_concat.pivot(index="country", columns="Years", values="Number").reset_index().rename_axis(None, axis=1).sort_values(by="country")
df_pivot.head(200)

,country,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Austria,1332.000000,1212.000000,1211.000000,1161.000000,1331.000000,1312.000000,1495.000000,1526.000000,1505.000000,NaN
1,Belgium,1482.000000,1563.000000,1556.000000,1671.000000,1858.000000,1861.000000,2026.000000,2131.000000,2412.000000,2503.0
2,Bulgaria,409.000000,314.000000,355.000000,419.000000,426.000000,449.000000,484.000000,487.000000,497.000000,448.0
3,Croatia,438.000000,600.000000,489.000000,526.000000,562.000000,626.000000,563.000000,585.000000,566.000000,NaN
4,Cyprus,28.000000,33.000000,30.000000,57.000000,69.000000,52.000000,59.000000,59.000000,56.000000,NaN
5,Czechia,409.000000,314.000000,355.000000,419.000000,426.000000,449.000000,484.000000,487.000000,497.000000,448.0
6,Denmark,409.000000,314.000000,355.000000,419.000000,426.000000,449.000000,484.000000,487.000000,497.000000,448.0
7,Estonia,144.000000,123.000000,164.000000,140.000000,157.000000,196.000000,170.000000,231.000000,203.000000,NaN
8,France,598.607843,619.352941,625.803922,637.886792,746.241935,780.677419,805.387097,902.589286,958.537037,551.5
9,Germany,853.285714,886.714286,883.142857,923.714286,1594.300000,1672.200000,1652.600000,1720.500000,1737.900000,NaN


In [325]:
df_pivot.to_csv("../data/out/processed_data.csv", index=False)